In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils
import geemask



__selection of reference point__

assume reference image\
assume roi to be specified in (an __integer__) number-of-pixels (size) in context of this image or projection

to obtain a symetrical roi, in this image (raster), around a (reference) point, :
- in case of an odd size, this reference point should be the center of a pixel
- in case of an even size, this reference point should be on a pixels border (intersection on grid)


target image can be warped to align with this roi by reprojecting the target image\
to the translated and rescaled projection of the reference image

assume target specifies its scale by stating its own (target) number-of-pixels in the reference roi.\
this number-of-pixels should be chosen
- large enough to avoid loss of data
- small enough to avoid redundant data
- (at least 4: however large the pixels are, they could always intersect in the roi)

e.g.: reference: S2 10m product, roi 32 pixels wide roi (hence approximatly 320m)\
consider target to be some PV 333m product.\
=> in the 320 x 320m roi, one could expect minimum 1, maximum 4 different PV 333m values\
=> roi ~32 ref pixels wide ~2 target pixels wide.

? heuristics: 
- choose target pixels ~  int( (ref pixels x ref res) / (target res) ) + 1 roughly (if more or less aligned)
- choose target pixels ~  int( (ref pixels x ref res) / (target res) ) + 2 safer (accounting for edges)



In [ ]:

def squarepixelboundsroi(eepoint, pixelsdiameter, eerefimage, verbose=False):
    size = round(pixelsdiameter)    #  "an integer" I said.
    size = max(size, 1)             #  preferably larger then 1 'reference image' pixel
    if (size %2) == 0:
        # even diameter
        eepoint = geeutils.pixelinterspoint(eepoint, eerefimage)
    else:
        # odd diameter
        eepoint = geeutils.pixelcenterpoint(eepoint, eerefimage)
    
    pixelsradius = size/2           # odd sizes: 1, 2, 3, ... - even sizes: 0.5, 1.5, 2.5, ...
    eeroi        = geeutils.squarerasterboundsroi(eepoint, pixelsradius, eerefimage)

    if verbose:
        print(f"squarepixelboundsroi pixelsdiameter({pixelsdiameter}) - pixelsradius({pixelsradius})")
        print(geeutils.szgeometryinfo(eepoint))
        print(geeutils.szgeometryinfo(eeroi))

    return eeroi, eepoint

def szpixelcount(eeimage, eegeometry):
    pixelcount = eeimage.select(0).unmask(sameFootprint=False).reduceRegion(ee.Reducer.count(), eegeometry)
    sz = f"pixelcount: {pixelcount.getInfo()}"
    return sz

__Proba V 333m with reference S2 10m__

In [ ]:
eepoint      = geeutils.tapspoint
eedate       = geeutils.fleecycloudsday

eerefimage     = geeutils.someS2ndviImageNear(eedate, eepoint)
refroidiameter = 32

eeroi, eeroicenterpoint = squarepixelboundsroi(eepoint, refroidiameter, eerefimage, verbose=False)

eeulx = eeroi.coordinates().flatten().get(0)
eeuly = eeroi.coordinates().flatten().get(1)



eesrcimage     = geeutils.somePV333ndviImageNear(eedate, eepoint)
dstroidiameter = 2

eedstimage = eesrcimage.reproject(eerefimage.projection()
                                  .translate(eeulx, eeuly)
                                  .scale(refroidiameter/dstroidiameter, refroidiameter/dstroidiameter))

print( szpixelcount(eerefimage, eeroi) )
print( szpixelcount(eesrcimage, eeroi) )
print( szpixelcount(eedstimage, eeroi) )
    
map = geemap.Map(height='600px')
map.centerObject(eepoint, 16)
map.addLayer(eesrcimage, {'min':0, 'max':1}, 'eesrcimage')
map.addLayer(eerefimage, {'min':0, 'max':1}, 'eerefimage')
map.addLayer(eedstimage, {'min':0, 'max':1}, 'eedstimage')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 2),            {'palette':'#0000ff'}, 'roi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#ffffff'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#00ff00'}, 'roicenterpoint')
map


__Sentinel 1 VV/VH bands with reference S2 10m__

In [ ]:
eepoint      = geeutils.tapspoint
eedate       = geeutils.fleecycloudsday

eerefimage     = geeutils.someS2ndviImageNear(eedate, eepoint)
refroidiameter = 5

eeroi, eeroicenterpoint = squarepixelboundsroi(eepoint, refroidiameter, eerefimage, verbose=False)

eeulx = eeroi.coordinates().flatten().get(0)
eeuly = eeroi.coordinates().flatten().get(1)



eesrcimage     = geeutils.someS1rviImageNear(eedate, eepoint)
dstroidiameter = 5

eedstimage = eesrcimage.reproject(eerefimage.projection()
                                  .translate(eeulx, eeuly)
                                  .scale(refroidiameter/dstroidiameter, refroidiameter/dstroidiameter))

print( szpixelcount(eerefimage, eeroi) )
print( szpixelcount(eesrcimage, eeroi) )
print( szpixelcount(eedstimage, eeroi) )
    
map = geemap.Map(height='600px')
map.centerObject(eepoint, 18)
map.addLayer(eesrcimage, {'min':0, 'max':1}, 'eesrcimage')
map.addLayer(eerefimage, {'min':0, 'max':1}, 'eerefimage')
map.addLayer(eedstimage, {'min':0, 'max':1}, 'eedstimage')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 2),            {'palette':'#0000ff'}, 'roi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#ffffff'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#00ff00'}, 'roicenterpoint')
map


__Sentinel 1 'angle' band with reference S2 10m__

In [ ]:
eepoint      = geeutils.tapspoint
eedate       = geeutils.fleecycloudsday

eerefimage     = geeutils.someS2ndviImageNear(eedate, eepoint)
refroidiameter = 128

eeroi, eeroicenterpoint = squarepixelboundsroi(eepoint, refroidiameter, eerefimage, verbose=False)

eeulx = eeroi.coordinates().flatten().get(0)
eeuly = eeroi.coordinates().flatten().get(1)



eesrcimage     = geeutils.someImageNear(geeutils.s1rbgImageCollection.select('angle'), eedate, eepoint)
dstroidiameter = 2

eedstimage = eesrcimage.reproject(eerefimage.projection()
                                  .translate(eeulx, eeuly)
                                  .scale(refroidiameter/dstroidiameter, refroidiameter/dstroidiameter))

print(geeutils.szestimatevaluesinfo(eedstimage.clip(eepoint.buffer(100000))))
print( szpixelcount(eerefimage, eeroi) )
print( szpixelcount(eesrcimage, eeroi) )
print( szpixelcount(eedstimage, eeroi) )
    
map = geemap.Map(height='600px')
map.centerObject(eepoint, 12)
map.addLayer(eesrcimage, {'min':30, 'max':38}, 'eesrcimage')
map.addLayer(eerefimage, {'min':0,  'max':1 }, 'eerefimage')
map.addLayer(eedstimage, {'min':30, 'max':38}, 'eedstimage')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 2),            {'palette':'#0000ff'}, 'roi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#ffffff'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#00ff00'}, 'roicenterpoint')
map


__Sentinel 2 SCL bands with reference S2 10m__

In [ ]:
eepoint      = geeutils.tapspoint
eedate       = geeutils.fleecycloudsday

eerefimage     = geeutils.someS2ndviImageNear(eedate, eepoint)
refroidiameter = 32

eeroi, eeroicenterpoint = squarepixelboundsroi(eepoint, refroidiameter, eerefimage, verbose=False)

eeulx = eeroi.coordinates().flatten().get(0)
eeuly = eeroi.coordinates().flatten().get(1)



eesrcimage     = geeutils.someImageNear(geeutils.s2sclImageCollection, eedate, eepoint)

dstroidiameter = 16

eedstimage = eesrcimage.reproject(eerefimage.projection()
                                  .translate(eeulx, eeuly)
                                  .scale(refroidiameter/dstroidiameter, refroidiameter/dstroidiameter))

print( szpixelcount(eerefimage, eeroi) )
print( szpixelcount(eesrcimage, eeroi) )
print( szpixelcount(eedstimage, eeroi) )
    
map = geemap.Map(height='400px')
map.centerObject(eepoint, 16)
map.addLayer(eesrcimage, geeutils.s2sclvisParams, 'eesrcimage')
map.addLayer(eerefimage, {'min':0,  'max':1 },    'eerefimage')
map.addLayer(eedstimage, geeutils.s2sclvisParams, 'eedstimage')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 2),            {'palette':'#0000ff'}, 'roi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#ffffff'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#00ff00'}, 'roicenterpoint')
map


__Sentinel 2 SCL ConvMask band with reference S2 10m__

In [ ]:
eepoint      = geeutils.bobspoint
eedate       = geeutils.fleecycloudsday

eerefimage     = geeutils.someS2ndviImageNear(eedate, eepoint)
refroidiameter = 32

eeroi, eeroicenterpoint = squarepixelboundsroi(eepoint, refroidiameter, eerefimage, verbose=False)

eeulx = eeroi.coordinates().flatten().get(0)
eeuly = eeroi.coordinates().flatten().get(1)


eesrcimage = geeutils.someImageNear(geeutils.s2sclImageCollection, eedate, eepoint)

def convmask(image):
    return ee.Image((geemask.ConvMask( [[2, 4, 5, 6, 7], [3, 8, 9, 10, 11]], [20*9, 20*101] ,[-0.057, 0.025] )
            .makemask(image)
            .rename('MASK')
            .copyProperties(image, ['system:id', 'system:time_start'])))

dstroidiameter = 16

eedstimage = convmask(eesrcimage).reproject(eerefimage.projection()
                                  .translate(eeulx, eeuly)
                                  .scale(refroidiameter/dstroidiameter, refroidiameter/dstroidiameter))

print( szpixelcount(eerefimage, eeroi) )
print( szpixelcount(eesrcimage, eeroi) )
print( szpixelcount(eedstimage, eeroi) )

eedstimage = eedstimage.updateMask(eedstimage.eq(1))
    
map = geemap.Map(height='400px')
map.centerObject(eepoint, 13)
map.addLayer(eesrcimage, geeutils.s2sclvisParams, 'eesrcimage')
map.addLayer(eerefimage, {'min':0,  'max':1 , 'palette':geeutils.ndvivisParamsPalette},    'eerefimage')
map.addLayer(eedstimage, {'min':0,  'max':1 },                                             'eedstimage')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 2),            {'palette':'#0000ff'}, 'roi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#ffffff'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#00ff00'}, 'roicenterpoint')
map
